In [2]:
import pandas as pd
import numpy as np

In [3]:
def format_stars(stars):
    stars = stars.split(', \'')
    stars = [star for star in stars if star != '' and star != '    Stars:\'']
    stars[0] = stars[0][2:]
    stars[-1] = stars[-1][:-2]

    dir_index = -1
    for j in range(len(stars)):
        stars[j] = stars[j].replace(', "', '')
        if 'Stars:\'' in stars[j]:
            stars[j] = stars[j].split(':\'')[1]
        
        if '|' in stars[j]:
            dir_index = j
    
    stars = stars[dir_index + 1:]

    return stars

def format_genres(genre):
    return genre.split(', ')

def format_years(year):
    if len(year) < 5 or year[1] not in ['1', '2']:
        return -1
    return int(year[1:5])

def set_is_tv(certificate):
    tv_certs = ['TV-Y', 'TV-Y7', 'TV-Y7-FV','TV-G','TV-PG','TV-14', 'TV-MA']

    if certificate in tv_certs:
        return True
    return False

def get_cleaned_dataframe():
    df = pd.read_csv('./n_movies.csv')
    df = df.drop(columns=['description', 'votes', 'duration'])
    df = df.dropna(subset=['stars'])
    df = df.drop_duplicates(subset=['title'])
    df = df.reset_index()
    df = df.fillna('')

    df['stars'] = df['stars'].apply(format_stars)
    df['genre'] = df['genre'].apply(format_genres)
    df['year'] = df['year'].apply(format_years)
    df['is_tv'] = df['certificate'].apply(set_is_tv)

    return df


In [4]:
df = get_cleaned_dataframe()

In [5]:
conda install pyvis

Channels:
 - defaults
 - conda-forge
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [11]:
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from pyvis.network import Network

# Network Visualization of Movies with shared actors

# Movie Network
movie_network = nx.MultiGraph()
tv_show_network = nx.MultiGraph()
movies = df.where(df['is_tv'] == False).dropna().reset_index(drop=True)
tv_shows = df.where(df['is_tv'] == True).dropna().reset_index(drop=True)

for i in tqdm(range(2000)):
    movie_network.add_node(movies.loc[i,'title'])

for i in tqdm(range(2000)):
    for j in range(i + 1, 2000):
        for actor in movies.loc[i, 'stars']:
            if actor in movies.loc[j, 'stars']:
                movie_network.add_edge(movies.loc[i, 'title'], movies.loc[j, 'title'])


# TV Show Network

for i in tqdm(range(200)):
    tv_show_network.add_node(tv_shows.loc[i,'title'])

for i in tqdm(range(200)):
    for j in range(i + 1, 200):
        for actor in tv_shows.loc[i, 'stars']:
            if actor in tv_shows.loc[j, 'stars']:
                tv_show_network.add_edge(tv_shows.loc[i, 'title'], tv_shows.loc[j, 'title'])
nt = Network("500px", "500px")
nt.from_nx(movie_network)
nt.show("nx.html")
'''
# smaller nodes and fonts
plt.figure(1, figsize=(15,15))
nx.draw_random(movie_network, node_size=50, with_labels=True)
plt.figure(2, figsize=(15,15))
nx.draw_random(tv_show_network, node_size=50, with_labels=True) 
plt.show()
'''

100%|████████████████████████████████████████| 200/200 [00:00<00:00, 576.00it/s]


'\n# smaller nodes and fonts\nplt.figure(1, figsize=(15,15))\nnx.draw_random(movie_network, node_size=50, with_labels=True)\nplt.figure(2, figsize=(15,15))\nnx.draw_random(tv_show_network, node_size=50, with_labels=True) \nplt.show()\n'